In [3]:
import tensorflow as tf
import numpy as np
import pretty_midi as midi
import os
import fluidsynth
import matplotlib.pyplot as plt
from tensorflow import keras 
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import time
import pandas as pd

In [4]:
TIME_STEPS = 35 
MIDI_PITCH_RANGE = 128

In [11]:
#midi_data is a list of PrettyMIDI objects
midi_data = []
directory = "data"

for filename in os.listdir(directory):
    midi_data.append(midi.PrettyMIDI(os.path.join(directory, filename)))
    

In [25]:
#Pull notes from midi file
velocities = []
pitches = []
starts = []
ends = []
durations = []
#Iterate through all midi objects, pull notes and zip together in a list 
for midi_obj in midi_data:
    piano = midi_obj.instruments[0]
    for note in piano.notes:
        velocities.append(note.velocity)
        pitches.append(note.pitch)
        starts.append(note.start)
        ends.append(note.end)
        durations.append(note.get_duration())

note_data_lists = list(zip(velocities, pitches, starts, ends, durations))

#Move zipped lists into dataframe and sort by start time so notes are chronological 
note_data = pd.DataFrame(note_data_lists, columns = ['Velocities', 'Pitches', 'Starts', 'Ends', 'Durations'])
note_data.sort_values(by = ["Starts"])


0          0.980469
1          1.065104
2          1.188802
3          0.993490
4          1.437500
            ...    
22292    442.621875
22293    441.944792
22294    443.776042
22295    443.769792
22296    443.794792
Name: Starts, Length: 22297, dtype: float64

,Velocities,Pitches,Starts,Ends,Durations
0,80,69,0.980469,1.074219,0.093750
13697,45,69,0.983073,2.623698,1.640625
14201,49,68,0.985417,1.478125,0.492708
3,57,45,0.993490,1.407552,0.414062
16236,66,67,1.000000,1.104167,0.104167
...,...,...,...,...,...
22293,62,37,441.944792,443.887500,1.942708
22292,74,65,442.621875,443.868750,1.246875
22295,79,66,443.769792,449.291667,5.521875
22294,71,54,443.776042,449.029167,5.253125
